# Using EcoFOCIpy to process raw field data

## DY2012 (Spring Mooring Cruise - Oscar Dyson)

## BTL Data + Nutrient Data

This is a streamlined version of generation routines to merge bottle data and Mordy Nut. Lab Nutrient Data for long term archive

In [21]:
import yaml
import glob
import pandas as pd
import os

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

In [3]:
sample_data_dir = '/Users/bell/ecoraid/2020/CTDcasts/dy2012/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [8]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
oxydatafile = sample_data_dir+'working/DiscreteOxygen/DY2012 Oxygen Data.txt' #<- point to cruise and process all files within
cruise_name = 'dy2012' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2012.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTDpOxy.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/CTD001.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2020/CTDcasts/dy2012/rawconverted/ctd014.btl
Processing /

## Load csv Oxygen File

In [11]:
oxy_data = pd.read_csv(oxydatafile,delimiter='\t')
oxy_data

,Cast,Niskin,"O2, uM/l"
0,2,2,271.625656
1,4,10,292.086083
2,8,10,289.943623
3,10,2,352.473353
4,16,10,305.062996
5,18,2,312.437279
6,21,10,295.776706
7,22,2,323.813606
8,24,2,317.877611
9,24,8,302.419710


## Merge Bottle and Nutrient Data but drop non nutrient vars?

In [12]:
keep_param = ['bottle','prdm']

for cast,cdata in cruise_data.items():
    try:
        matchcast = int((cast.split('.')[0]).lower().split('ctd')[-1])
        cruise_data[cast] = pd.merge(oxy_data[oxy_data['Cast']==matchcast],cdata.reset_index()[keep_param],right_on='bottle',left_on='Niskin').set_index('bottle').drop(columns=['Cast'])
    except:
        continue

## Add Deployment meta information

In [13]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [18]:
with open(inst_meta_file) as file:
    inst_oxy_config = yaml.full_load(file)

## Add institutional meta-information


In [19]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [26]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            'O2 (uM)':'O2',
                            'O2, uM/l':'O2',
                            'Niskin':'BTLID',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_oxy_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='excellent') #<- options are unknown, excellent, probably good, mixed, unqcd

        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast.lower().split('d')[-1].split('.')[0].zfill(3)+'_oxy.nc',format="NETCDF3_CLASSIC")
    except KeyError:
        print(f'Skipping {cast}')
    except RuntimeError:
        print(f'Skipping & Removing {cast}')
        os.remove(path=cruise_name+'c'+cast.lower().split('d')[-1].split('.')[0].zfill(3)+'_oxy.nc')

Skipping & Removing CTD001.btl
Skipping & Removing ctd003.btl
Skipping & Removing ctd005.btl
Skipping & Removing ctd006.btl
Skipping & Removing ctd007.btl
Skipping & Removing ctd009.btl
Skipping & Removing ctd011.btl
Skipping & Removing ctd012.btl
Skipping & Removing ctd014.btl
Skipping & Removing ctd015.btl
Skipping & Removing ctd017.btl
Skipping & Removing ctd019.btl
Skipping & Removing ctd020.btl
Skipping & Removing ctd023.btl
Skipping & Removing ctd025.btl
Skipping & Removing ctd026.btl
Skipping & Removing ctd028.btl
Skipping & Removing ctd029.btl
Skipping & Removing ctd031.btl
Skipping ctd031a.btl
Skipping & Removing ctd032.btl
Skipping & Removing ctd035.btl
Skipping & Removing ctd036.btl
Skipping & Removing ctd039.btl
Skipping & Removing ctd040.btl
Skipping & Removing ctd041.btl
Skipping & Removing ctd042.btl
Skipping & Removing ctd045.btl
Skipping & Removing ctd046.btl
Skipping & Removing ctd049.btl
Skipping & Removing ctd050.btl
Skipping ctd051a.btl
Skipping & Removing ctd052.b

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history

In [27]:
 cruise_data[cast]

,BTLID,O2,pressure
bottle,,,
10.0,10,284.003111,2.561
